# ReAct (Reason + Act) 智能体实战

本笔记本演示了如何实现经典的 **ReAct** 范式。ReAct 的核心思想是模拟人类解决问题的方式：将 **推理 (Reasoning)** 与 **行动 (Acting)** 结合起来，形成一个“思考-行动-观察”的循环。

### 核心流程：
1. **Thought (思考)**：分析当前情况，决定下一步做什么。
2. **Action (行动)**：调用外部工具执行操作。
3. **Observation (观察)**：获取工具执行后的反馈，并更新记忆。

### C/C++ 类比：
这就像是一个 **带有外部 API 调用的有限状态机 (FSM)**，其状态转移逻辑（Next State Logic）不是硬编码的，而是由 LLM 在运行时动态计算决定的。

In [8]:
import re
import os
from dotenv import load_dotenv

# 确保能导入同一目录下的 llm_client 和 tools
# 注意：在 Notebook 中，如果修改了 llm_client.py，需要重启 Kernel 才能生效。
from llm_client import HelloAgentsLLM
from tools import ToolExecutor, search

# 加载 .env 配置
load_dotenv(dotenv_path="../../.env")

True

In [ ]:
import json

# ========== V2 提示词：要求模型输出 JSON 格式 ==========
REACT_PROMPT_TEMPLATE = """
请注意，你是一个有能力调用外部工具的智能助手。

可用工具如下：
{tools}

你必须严格以 JSON 格式回应，不要输出任何其他文字。JSON 格式如下：
{{
    "thought": "你的思考过程",
    "action": "工具名称（如 Search）或 Finish",
    "action_input": "传给工具的参数，或最终答案"
}}

规则：
1. 当你需要使用工具时，"action" 填写工具名称，"action_input" 填写查询内容。
2. 当你已经获得足够信息可以回答时，"action" 填 "Finish"，"action_input" 填最终答案。
3. 只输出 JSON，不要在 JSON 前后添加任何解释文字或 Markdown 标记。

现在，请开始解决以下问题：
Question: {question}
History: {history}
"""

class ReActAgent:
    def __init__(self, llm_client: HelloAgentsLLM, tool_executor: ToolExecutor, max_steps: int = 5):
        self.llm_client = llm_client
        self.tool_executor = tool_executor
        self.max_steps = max_steps
        self.history = []

    def run(self, question: str):
        self.history = []
        current_step = 0

        while current_step < self.max_steps:
            current_step += 1
            print(f"\n--- 第 {current_step} 步 ---")

            tools_desc = self.tool_executor.getAvailableTools()
            history_str = "\n".join(self.history)
            prompt = REACT_PROMPT_TEMPLATE.format(tools=tools_desc, question=question, history=history_str)

            messages = [{"role": "user", "content": prompt}]
            response_text = self.llm_client.think(messages=messages)
            if not response_text: break

            # 使用 JSON 解析
            thought, action_name, action_input = self._parse_json_output(response_text)
            
            if thought: print(f"🤔 思考: {thought}")
            if not action_name: 
                print("警告：未能解析出有效的 Action。"); break
            
            # 判断是否结束
            if action_name == "Finish":
                print(f"🎉 最终答案: {action_input}")
                return action_input
            
            # 执行工具
            print(f"🎬 行动: {action_name}[{action_input}]")
            tool_function = self.tool_executor.getTool(action_name)
            observation = tool_function(action_input) if tool_function else f"错误：未找到工具 '{action_name}'。"
            
            print(f"👀 观察: {observation}")
            self.history.append(f"Action: {action_name}[{action_input}]")
            self.history.append(f"Observation: {observation}")

        print("已达到最大步数，流程终止。")
        return None

    def _parse_json_output(self, text: str):
        """
        V2 解析器：使用 json.loads 替代正则。
        """
        try:
            # 兼容模型可能在 JSON 外包裹 ```json ``` 的情况
            if "```json" in text:
                text = text.split("```json")[1].split("```")[0]
            elif "```" in text:
                text = text.split("```")[1].split("```")[0]
            
            data = json.loads(text.strip())
            thought = data.get("thought", "")
            action = data.get("action", "")
            action_input = data.get("action_input", "")
            return thought, action, action_input
        except (json.JSONDecodeError, KeyError, IndexError) as e:
            print(f"❌ JSON 解析失败: {e}")
            print(f"原始输出: {text[:200]}")
            return None, None, None

In [10]:
# 实例化与运行
# 类比 C++ 的 main 函数入口

llm = HelloAgentsLLM()
tool_executor = ToolExecutor()

# 注册搜索工具
search_desc = "一个网页搜索引擎。当你需要回答关于时事、事实以及在你的知识库中找不到的信息时，应使用此工具。"
tool_executor.registerTool("Search", search_desc, search)

# 初始化 Agent
agent = ReActAgent(llm_client=llm, tool_executor=tool_executor)

# 提出一个需要实时搜索的问题
question = "apple最新的手机是哪一款？它的主要卖点是什么？"

print("--- 开始运行 ReAct Agent ---")
agent.run(question)

工具 'Search' 已注册。
--- 开始运行 ReAct Agent ---

--- 第 1 步 ---
🧠 正在调用 gpt-4o 模型...
✅ 大语言模型响应成功:
{
    "thought": "我需要查找苹果最新发布的手机型号及其主要卖点。",
    "action": "Search",
    "action_input": "苹果最新的手机型号及主要卖点"
}
🤔 思考: 我需要查找苹果最新发布的手机型号及其主要卖点。
🎬 行动: Search[苹果最新的手机型号及主要卖点]
🔍 正在执行 [SerpApi] 网页搜索: 苹果最新的手机型号及主要卖点
👀 观察: [1] 2026年苹果iPhone手机推荐哪一款性价比最高？（8000字选 ...
最新的iPhone共有iPhone 17、iPhone Air、iPhone 17 Pro以及iPhone 17 Pro Max四款机型。其中，属于基本款的iPhone 17配备6.3英寸120Hz高刷显示屏和A19芯片，相对平价且已 ...

[2] 古尔曼曝苹果将掀新品发布潮： iPhone 17e、新iPad合力猛攻 ...
古尔曼称，即将发布的iPhone 17e将用来取代一年前推出的iPhone 16e，这款入门款机型的升级包括搭载与iPhone 17 相同的A19芯片，并加入MagSafe磁吸充电。 同时 ...

[3] 一图看懂苹果2025秋季新品发布会，八大新品卖点价格全汇总
本次发布会上，苹果带来了iPhone17、iPhone 17 Pro/Pro Max以及全新系列iPhone Air，还有AirPods Pro 3、Apple Watch Series 11、Apple Watch SE 3以及Apple ...

--- 第 2 步 ---
🧠 正在调用 gpt-4o 模型...
✅ 大语言模型响应成功:
```json
{
    "thought": "根据搜索结果，苹果最新的手机是iPhone 17系列，包括iPhone 17、iPhone 17 Pro、iPhone 17 Pro Max以及全新系列iPhone Air。接下来总结其主要卖点。",
    "action": "Finish",
    "action_input": "苹果最新的手机是iPho

'苹果最新的手机是iPhone 17系列，包括iPhone 17、iPhone 17 Pro、iPhone 17 Pro Max以及全新系列iPhone Air。主要卖点包括：iPhone 17配备6.3英寸120Hz高刷显示屏和A19芯片；iPhone 17e作为入门款，搭载与iPhone 17相同的A19芯片，并加入MagSafe磁吸充电；iPhone 17 Pro和Pro Max则可能在性能和摄像头等方面有更高配置。'